In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

C:\Users\Abhishek\Anaconda2\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [15]:
date_columns = ['Internship_deadline', 'Earliest_Start_Date', 'Start Date', 
                'End Date', 'Start_Date']

train_merged = pd.read_csv('../data/train_merged.csv', parse_dates=date_columns)
test_merged = pd.read_csv('../data/test_merged.csv', parse_dates=date_columns)

## Training dataset

In [3]:
train_merged.head()

,Internship_ID,Student_ID,Earliest_Start_Date,Expected_Stipend,Minimum_Duration,Preferred_location,Is_Part_Time,Is_Shortlisted,Internship_Profile,Skills_required,...,Performance_10th,Experience_Type,Profile,Location,Start Date,End Date,num_experience,num_exp_in_job,num_awards,num_previous_internships
0,8161,78663553,2015-01-03,2-5K,3,NaN,0,0,Voice Over Movie,NaN,...,80.4,internship,About C,JBEI,08-01-2013,08-02-2013,1,0,0,1
1,4977,7695797,2014-12-19,5-10K,2,IHFG,1,0,Social Media Marketing & Design,NaN,...,82.0,NaN,NaN,IIGB,NaN,NaN,1,0,0,0
2,10271,78663092,2015-01-06,5-10K,6,NaN,0,0,Java Development,NaN,...,81.4,academic_project,NaN,JBEI,11-01-2014,02-05-2014,5,0,0,0
3,7393,7708503,2014-12-03,2-5K,1,IHFG,0,0,Fashion Curator,NaN,...,91.0,NaN,NaN,IIGB,NaN,NaN,1,0,0,0
4,11125,78659782,2015-01-02,10K+,6,NaN,1,0,Business Development,"Communication,Business Development (Sales)",...,50.0,NaN,NaN,IIGB,NaN,NaN,1,0,0,0


## Test set

In [4]:
test_merged.head()

,Internship_ID,Student_ID,Earliest_Start_Date,Expected_Stipend,Minimum_Duration,Preferred_location,Is_Part_Time,Internship_Profile,Skills_required,Internship_Type,...,Performance_10th,Experience_Type,Profile,Location,Start Date,End Date,num_experience,num_exp_in_job,num_awards,num_previous_internships
0,9341,7677714,25-Jan-15,2-5K,2,IIBD,0,Content Writer,NaN,regular,...,88,NaN,NaN,IIGB,NaN,NaN,1,0,0,0
1,6426,7696418,21-Jan-15,5-10K,5,NaN,0,Android Development,NaN,regular,...,60,internship,NETWORK PLANNING,IIDB,20-06-2013,19-08-2013,2,0,0,1
2,5812,78666725,09-Jan-15,10K+,5,NaN,1,Recruitment,NaN,regular,...,57,NaN,NaN,IIGB,NaN,NaN,1,0,0,0
3,10711,7705454,11-Jan-15,No Expectations,2,NaN,1,Content Development,NaN,virtual,...,74,training,NaN,JBFA,06-05-2013,20-05-2013,1,0,0,0
4,5880,7711946,15-Jan-15,No Expectations,2,IHFG,1,Web Development,web development,regular,...,10,academic_project,Stock Analyst,JEJJ,06-03-2014,12-01-2015,6,0,2,1


## Class Distribution

In [5]:
train_merged.Is_Shortlisted.value_counts()

0    168003
1     24579
Name: Is_Shortlisted, dtype: int64

## Exploratory Data Analysis

In [16]:
train_merged.loc[:, 'Earliest_Start_Date_year'] = train_merged.Earliest_Start_Date.dt.year
train_merged.loc[:, 'Earliest_Start_Date_month'] = train_merged.Earliest_Start_Date.dt.month
train_merged.loc[:, 'Earliest_Start_Date_day'] = train_merged.Earliest_Start_Date.dt.day

test_merged.loc[:, 'Earliest_Start_Date_year'] = test_merged.Earliest_Start_Date.dt.year
test_merged.loc[:, 'Earliest_Start_Date_month'] = test_merged.Earliest_Start_Date.dt.month
test_merged.loc[:, 'Earliest_Start_Date_day'] = test_merged.Earliest_Start_Date.dt.day

train_merged.loc[:, 'Internship_deadline_year'] = train_merged.Internship_deadline.dt.year
train_merged.loc[:, 'Internship_deadline_month'] = train_merged.Internship_deadline.dt.month
train_merged.loc[:, 'Internship_deadline_day'] = train_merged.Internship_deadline.dt.day

test_merged.loc[:, 'Internship_deadline_year'] = test_merged.Internship_deadline.dt.year
test_merged.loc[:, 'Internship_deadline_year'] = test_merged.Internship_deadline.dt.month
test_merged.loc[:, 'Internship_deadline_year'] = test_merged.Internship_deadline.dt.day

### How do the acceptance rate change during different years ?

In [17]:
train_merged.groupby(['Earliest_Start_Date_year', 'Is_Shortlisted']).size()

Earliest_Start_Date_year  Is_Shortlisted
2014                      0                 106197
                          1                  17865
2015                      0                  61806
                          1                   6714
dtype: int64

In [22]:
train_merged.groupby(['Earliest_Start_Date_month', 'Is_Shortlisted']).size().unstack(1).fillna(0)

Is_Shortlisted,0,1
Earliest_Start_Date_month,,
1,52354,6098
2,2924,237
3,575,35
4,1963,127
5,2686,150
6,1101,61
7,126,2
8,25,0
9,6,0


In [23]:
train_merged.groupby(['Internship_deadline_year', 'Is_Shortlisted']).size()

Internship_deadline_year  Is_Shortlisted
2014                      0                 111844
                          1                  16250
2015                      0                  56159
                          1                   8329
dtype: int64

In [25]:
train_merged.groupby(['Internship_deadline_month', 'Is_Shortlisted']).size().unstack(1).fillna(0)

Is_Shortlisted,0,1
Internship_deadline_month,,
1,20615,3075
2,3288,549
3,1359,160
4,2454,474
5,5862,1142
6,8423,1423
7,5261,1078
8,6908,886
9,8005,1471


In [ ]:
train_merged.loc[:, 'date_diff'] = ((train_merged.Internship_deadline - train_merged.Earliest_Start_Date) / np.timedelta64(1, 'D')).astype(int)

In [ ]:
# Relationship between difference between internship deadline vs whether you are shorlisted or not

sns.FacetGrid(train_merged, hue="Is_Shortlisted", size=5) \
   .map(plt.hist, "date_diff") \
   .add_legend()

In [ ]:
## Relationship between expected stipend and Is_Shortlisted

train_merged.loc[:, ['Stipend_Type', 'Is_Shortlisted']].head()

In [ ]:
train_merged.Expected_Stipend.value_counts()

In [ ]:
def salary_mapping(salary):
    if salary < 2000:
        return 'No Expectations'
    elif salary >= 2000 and salary < 5000:
        return '2-5K'
    elif salary >= 5000 and salary < 10000:
        return '5-10K'
    else:
        return '10K+'

In [44]:
train_merged.Stipend1 = train_merged.Stipend1.fillna(train_merged.Stipend1.mean())

In [ ]:
train_merged.loc[:, 'Stipend_level'] = train_merged.Stipend1.map(salary_mapping)

In [ ]:
def check_if_expectations_match(row):
    expected_stipend = row['Expected_Stipend']
    stipend_level = row['Stipend_level']
    
    if expected_stipend == 'No Expectations':
        return 1
    elif expected_stipend == '2-5K':
        if stipend_level in ['2-5K', '5-10K', '10K+']:
            return 1
        else:
            return 0
    elif expected_stipend == '5-10K':
        if stipend_level in ['5-10K', '10K+']:
            return 1
        else:
            return 0
    elif stipend_level == '10K+':
        if stipend_level == '10K+':
            return 1
        else:
            return 0

train_merged.loc[:, 'expectations_match'] = train_merged[['Expected_Stipend', 'Stipend_level']].apply(check_if_expectations_match, axis=1)

In [ ]:
train_merged.groupby(['expectations_match', 'Is_Shortlisted']).size()

** This is a potential feature **

In [ ]:
train_merged.loc[:, 'normalized_stipend_1'] = np.log10(train_merged.Stipend1 + 1)

In [ ]:
sns.FacetGrid(train_merged, hue="Is_Shortlisted", size=5) \
   .map(plt.hist, "normalized_stipend_1") \
   .add_legend()

In [ ]:
def check_if_locations_match(row):
    internship_location = row['Internship_Location']
    student_location = row['Location']
    
    return int(internship_location == student_location)

In [ ]:
train_merged.loc[:, 'location_match'] = train_merged[['Internship_Location', 'Location']].apply(check_if_locations_match, axis=1)

In [ ]:
train_merged.groupby(['location_match', 'Is_Shortlisted']).size()

In [ ]:
train_merged.loc[:, ['Internship_Location', 'Is_Shortlisted']].head()

In [ ]:
print 'Number of levels of Internship location ', len(train_merged.Internship_Location.unique())

In [ ]:
len(train_merged.Location.unique())

In [ ]:
train_merged.Internship_Location.value_counts()

In [ ]:
train_merged.groupby(['Internship_Location', 'Is_Shortlisted']).size()

In [ ]:
train_merged.Skills_required.unique()

In [26]:
train_merged.select_dtypes(include=['object']).columns

Index([u'Earliest_Start_Date', u'Expected_Stipend', u'Preferred_location',
       u'Internship_Profile', u'Skills_required', u'Internship_Type',
       u'Internship_Location', u'Internship_category', u'Stipend_Type',
       u'Internship_deadline', u'Start_Date', u'Institute_Category',
       u'Institute_location', u'hometown', u'Degree', u'Stream',
       u'Current_year', u'Experience_Type', u'Profile', u'Location',
       u'Start Date', u'End Date'],
      dtype='object')

## Pivot Tables

In [30]:
features = ['Expected_Stipend', 'Stipend_Type', 'Is_Shortlisted', 'Internship_Location', 'Experience_Type',
            'Institute_Category', 'Stipend1', 'Degree', 'Stream', 'Minimum_Duration', 'Current_year', 'Preferred_location']

In [14]:
train_merged[features].pivot_table(index=['Is_Shortlisted'],
                                   columns=['Stipend_Type', 'Current_year'],
                                   values=['Stipend1'],
                                   aggfunc='mean', margins=False, fill_value=0)

Stipend1                                         \
Stipend_Type          fixed                                          
Current_year              1            2            3            4   
Is_Shortlisted                                                       
0               7579.383768  7515.094193  8174.521115  9244.285303   
1               5630.111821  5845.255474  6536.957265  7637.785800   

                                                                          \
Stipend_Type                                    performance                
Current_year              5 already a graduate            1            2   
Is_Shortlisted                                                             
0               9419.234194        9178.604877  2116.076549  2334.481667   
1               7805.755396        7230.996979  2209.172199  2082.693878   

                                                ...                          \
Stipend_Type                                    ...         unpaid            
Current_year              3            4        ...              3     4  5   
Is_Shortlisted                                  ...                           
0               3449.671517  4202.705403        ...           5000  3000  0   
1               2602.481651  2540.230159        ...           5000     0  0   

                                                                          \
Stipend_Type                          variable                             
Current_year   already a graduate            1            2            3   
Is_Shortlisted                                                             
0                            3000  5584.469540  5403.701492  5856.074422   
1                            5000  5015.813351  4937.581583  5317.309404   

                                                             
Stipend_Type                                                 
Current_year              4            5 already a graduate  
Is_Shortlisted                                               
0               6579.529857  6893.594646        6836.582812  
1               6197.576173  7321.428571        5924.171330  

[2 rows x 24 columns]

In [12]:
train_merged[features].pivot_table(index=['Is_Shortlisted'],
                                   columns=['Stipend_Type', 'Experience_Type'],
                                   values=['Minimum_Duration'],
                                   aggfunc='mean', margins=False, fill_value=0)

Minimum_Duration                                           \
Stipend_Type               fixed                                            
Experience_Type academic_project     award internship       job     other   
Is_Shortlisted                                                              
0                       4.487255  3.596313   3.534568  4.436526  3.384961   
1                       4.195681  3.242424   3.224764  4.060332  3.070588   

                                                                             \
Stipend_Type                                                    performance   
Experience_Type participation       por training  workshop academic_project   
Is_Shortlisted                                                                
0                    3.090769  2.940144  4.39824  3.277799         3.210710   
1                    2.800797  2.734990  3.84979  2.958065         4.204301   

                   ...                                                     \
Stipend_Type       ...       unpaid         variable                        
Experience_Type    ...     workshop academic_project     award internship   
Is_Shortlisted     ...                                                      
0                  ...     2.278481         4.173983  3.421344   3.400941   
1                  ...     1.787879         4.345535  3.514905   3.355532   

                                                                       \
Stipend_Type                                                            
Experience_Type       job     other participation       por  training   
Is_Shortlisted                                                          
0                4.366160  3.184024      3.026016  2.874465  4.187304   
1                4.278919  3.070175      3.187668  2.858169  4.254237   

                           
Stipend_Type               
Experience_Type  workshop  
Is_Shortlisted             
0                3.103653  
1                3.249307  

[2 rows x 36 columns]

In [29]:
train_merged.groupby(['Experience_Type', 'Is_Shortlisted']).size()

Experience_Type   Is_Shortlisted
academic_project  0                 25567
                  1                  3567
award             0                  5177
                  1                   856
internship        0                 29928
                  1                  5758
job               0                 12215
                  1                  1810
other             0                  2078
                  1                   409
participation     0                  4647
                  1                   766
por               0                  6166
                  1                  1241
training          0                 18830
                  1                  2339
workshop          0                  6220
                  1                   806
dtype: int64

In [32]:
train_merged[features].pivot_table(index=['Is_Shortlisted'],
                                   columns=['Expected_Stipend'],
                                   values=['Stipend1'],
                                   aggfunc='mean', margins=False, fill_value=0)

Stipend1                                          
Expected_Stipend          10K+         2-5K        5-10K No Expectations
Is_Shortlisted                                                          
0                 11121.041976  4350.442433  6638.842567     6272.084319
1                  9507.761377  3601.188320  6008.770300     4582.913441

In [34]:
train_merged.pivot_table(index=['Is_Shortlisted'],
                                   columns=['Year_of_graduation'],
                                   values=['Stipend1'],
                                   aggfunc='mean', margins=False, fill_value=0)

Stipend1                                         \
Year_of_graduation          2001         2009         2010         2011   
Is_Shortlisted                                                            
0                    9454.545455  7552.423251  7042.297491  7989.000000   
1                   10000.000000  5627.873134  5463.709677  6502.343396   

                                                                        \
Year_of_graduation         2012         2013         2014         2015   
Is_Shortlisted                                                           
0                   7663.992942  7476.226672  7453.349203  7711.746458   
1                   6153.908795  6284.323427  5777.529893  6392.268165   

                                                                        \
Year_of_graduation         2016         2017         2018         2019   
Is_Shortlisted                                                           
0                   5967.582057  5160.863249  4523.549689  5302.081218   
1                   4995.858279  4331.715686  4001.512524  3982.631579   

                                  
Year_of_graduation          2020  
Is_Shortlisted                    
0                    6317.241379  
1                   15000.000000

In [44]:
train_merged[train_merged.Year_of_graduation==2016].Stipend1.mean()

5832.597621986875

## Look at the degrees

In [56]:
train_merged.Degree.value_counts()

B.Tech                                                      60082
B.E                                                         29549
MCA                                                         21449
MBA                                                          9004
B.Com                                                        5208
B.Com (Hons.)                                                3632
Post Graduate Dimploma in Management                         3189
B.Sc                                                         3167
Bachelor of Business Admininstration                         2752
M.Tech                                                       2691
BCA                                                          1890
B.A (Hons.)                                                  1871
M.Sc                                                         1793
B.Sc (Hons.)                                                 1558
B.M.S.                                                       1379
B.A. Progr

In [72]:
degree_popularity = train_merged.groupby(['Degree', 'Is_Shortlisted']).size().unstack(1).fillna(0)
degree_popularity.loc[:, 'percentage_accepted'] = (degree_popularity[1] / (degree_popularity[0]+ degree_popularity[1]) * 1.) * 100.

In [74]:
degree_popularity.sort_values(by=[1,'percentage_accepted'], ascending=False)

Is_Shortlisted,0,1,percentage_accepted
Degree,,,
B.Tech,53006,7076,11.777238
B.E,26341,3208,10.856543
MCA,19366,2083,9.711408
MBA,7842,1162,12.905375
B.Com,4454,754,14.477727
B.Com (Hons.),3012,620,17.070485
Bachelor of Business Admininstration,2326,426,15.479651
Post Graduate Dimploma in Management,2790,399,12.511759
B.Sc,2770,397,12.535523


In [83]:
train_merged.groupby(['Stream', 'Is_Shortlisted']).size().unstack(1).fillna(0).sort_values(by=1, ascending=False)

Is_Shortlisted,0,1
Stream,,
Computer Science & Engineering,23421,3012
Computer Science,16265,1976
Commerce,8519,1522
Electronics and Communication Engineering,11915,1406
Information Technology,9085,1291
Computer Application,9932,1188
Marketing,4828,936
Mechanical Engineering,6985,681
Commerce With Maths,3322,662


In [85]:
train_merged.Stipend_Type.value_counts()

variable       93315
fixed          84740
performance    12267
unpaid          2260
Name: Stipend_Type, dtype: int64